In [291]:
from tqdm.auto import tqdm 
import pandas as pd
import numpy as np
import xarray as xr
import glob
%matplotlib inline

import cartopy.crs as ccrs
import matplotlib.pyplot as plt

In [39]:
datapath="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/Data/ECMWF/ERA5_25kmx3hr/"
path="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/besttracks/"
output="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/create_ts/outputs/"
target="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/create_ts/outputs/targets/"

In [292]:
track = sorted(glob.glob(path+'*2001*.csv'))

In [293]:
tracksDF = pd.read_csv(track[0])

In [294]:
tracksDF['name'].unique()

array(['CIMARON', 'CHEBI', 'UTOR', 'KONG-REY', 'TORAJI', 'MAN-YI',
       'PABUK', 'WUTIP', 'FITOW', 'DANAS', 'NARI', 'VIPA', 'FRANCISCO',
       'LEKIMA', 'KROSA', 'HAIYAN', 'PODUL', 'LINGLING', 'FAXAI'],
      dtype=object)

In [295]:
stormnames = list(tracksDF['name'].unique())

In [296]:
[str(tracksDF[tracksDF['name']==stormnames[0]].time[i]).split(':')[0] for i in range(len(tracksDF[tracksDF['name']==stormnames[0]].time))]

['2001-05-05',
 '2001-05-05',
 '2001-05-05',
 '2001-05-05',
 '2001-05-05',
 '2001-05-05',
 '2001-05-05',
 '2001-05-05',
 '2001-05-06',
 '2001-05-06',
 '2001-05-06',
 '2001-05-06',
 '2001-05-06',
 '2001-05-06',
 '2001-05-06',
 '2001-05-06',
 '2001-05-07',
 '2001-05-07',
 '2001-05-07',
 '2001-05-07',
 '2001-05-07',
 '2001-05-07',
 '2001-05-07',
 '2001-05-07',
 '2001-05-08',
 '2001-05-08',
 '2001-05-08',
 '2001-05-08',
 '2001-05-08',
 '2001-05-08',
 '2001-05-08',
 '2001-05-08',
 '2001-05-09',
 '2001-05-09',
 '2001-05-09',
 '2001-05-09',
 '2001-05-09',
 '2001-05-09',
 '2001-05-09',
 '2001-05-09',
 '2001-05-10',
 '2001-05-10',
 '2001-05-10',
 '2001-05-10',
 '2001-05-10',
 '2001-05-10',
 '2001-05-10',
 '2001-05-10',
 '2001-05-11',
 '2001-05-11',
 '2001-05-11',
 '2001-05-11',
 '2001-05-11',
 '2001-05-11',
 '2001-05-11',
 '2001-05-11',
 '2001-05-12',
 '2001-05-12',
 '2001-05-12',
 '2001-05-12',
 '2001-05-12',
 '2001-05-12',
 '2001-05-12',
 '2001-05-12',
 '2001-05-13',
 '2001-05-13',
 '2001-05-

In [297]:
#function
def output_indices(TCtrack=None,ERA5date=None,ERA5hour=None):
    allindices = []
    for timeidx in range(len(TCtrack)):#len(track['time'])):
        datetrack,hourtrack = TCtrack['time'][timeidx].split(':')[0],TCtrack['time'][timeidx].split(':')[1][0:2]
        ####################################################################################################
        # Find the indices in ERA5 data with the same date as track
        ####################################################################################################
        dateind = []
        for ind,obj in enumerate(ERA5date):
            if obj==datetrack:
                dateind.append(ind)
        del ind,obj
        hourind = []
        hourextract = ERA5hour[int(np.min(np.asarray(dateind))):int(np.max(np.asarray(dateind)))+1]
        for ind,obj in enumerate(hourextract):
            if obj==hourtrack:            
                hourind.append(ind)
        allindices.append((int(np.min(np.asarray(dateind))),int(hourind[0])))
    return allindices

def extract_var(dataset=None,var='var138',indices=None):
    extractedvar = []
    for i in (range(len(indices))):
        realindex = indices[i][0]+indices[i][1]
        extractedvar.append(dataset[var][int(realindex),...].data)
    return np.asarray(extractedvar)

def largearea(dataset=None,invar=None,indices=None):
    if len(invar.shape) != 3:
        invar = np.squeeze(invar)
    ds = xr.Dataset(
    data_vars=dict(variable=(["time","lat","lon"], invar)),#mysvar[0])),
    coords=dict(lat=(["lat"], dataset.lat.data),lon=(["lon"], dataset.lon.data),time=(["time"], np.linspace(0,len(indices)-1,len(indices)))),
    attrs=dict(description="coords with matrices"),)
    
    var_out=np.zeros((len(indices),64,64))
    for it in range(len(indices)):
        latn, lats, lone, lonw = tc_orad[it,:]
        try:
            var_out[it,:,:]=ds['variable'][it,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))
        except:
            var_out[it,:,:]=ds['variable'][it,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))[0:64,0:64]
    return var_out

def largearea_withpres(dataset=None,invar=None,indices=None):
    ds = xr.Dataset(
    data_vars=dict(variable=(["time","plev","lat","lon"], invar)),#mysvar[0])),
    coords=dict(lat=(["lat"], dataset.lat.data),lon=(["lon"], dataset.lon.data),time=(["time"], np.linspace(0,len(indices)-1,len(indices))),
               plev=(["plev"],dataset.plev.data)),
    attrs=dict(description="coords with matrices"),)
    var_out=np.zeros((len(indices),len(dm1.plev.data),64,64))
    for it in range(len(indices)):
        latn, lats, lone, lonw = tc_orad[it,:]
        for ip in range(len(dm1.plev.data)):
            try:
                var_out[it,ip,:,:]=ds['variable'][it,ip,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))
            except:
                var_out[it,ip,:,:]=ds['variable'][it,ip,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))[0:64,0:64]
    return var_out 

In [298]:
###
def createmask(dm=None,irad=None,orad=None,lonselect=None,latselect=None):
    mask = []
    for ti in range(len(tc_irad)):
        lonselect = dm.lon.sel(lon=slice(orad[ti,:][2],orad[ti,:][3])).data
        latselect = np.flipud(dm.lat.sel(lat=slice(orad[ti,:][1],orad[ti,:][0])).data)
        if (lonselect.shape != 64) or (latselect.shape != 64):
            lon2d,lat2d = np.meshgrid(lonselect[0:64],latselect[0:64])
        else:
            lon2d,lat2d = np.meshgrid(lonselect,latselect)
        #############################################################################################
        latcriteria = np.logical_and(lat2d>irad[ti][0],lat2d<irad[ti][1])
        loncriteria = np.logical_and(lon2d>irad[ti][2],lon2d<irad[ti][3])
        allcriteria = np.logical_and(loncriteria,latcriteria)
        mask.append(allcriteria)
    return mask



In [299]:
def readyear_automatic(year=None):
    dm2 = xr.open_dataset(datapath+'/slev_vars/svars_'+str(year)+'.nc')
    #tracklist = sorted(glob.glob('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/besttracks/nio/*_'+str(year)+'*'))
    era5_date = [str(dm2.time[i].data).split('T')[0] for i in range(len(dm2.time))]
    era5_hour = [str(dm2.time[i].data).split('T')[1][0:2] for i in range(len(dm2.time))]
    return dm2,era5_date,era5_hour

dm2,era5_date,era5_hour = readyear_automatic(2001)

In [300]:
TCs_ts = []
for TCobj in tqdm(stormnames):
    track=tracksDF[tracksDF['name']==TCobj].reset_index()
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mysvar = [extract_var(dataset=dm2,var=obj,indices=indices_store) for obj in (list(dm2.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2
    
    tc_orad=np.empty((len(indices_store),4))
    tc_orad[:,0] = pos[:,0]-8
    tc_orad[:,1] = pos[:,0]+8
    tc_orad[:,2] = pos[:,1]-8
    tc_orad[:,3] = pos[:,1]+8
    ###########################################################################
    smallsvarout = [largearea(dm2,mysvar[i],indices_store) for i in (range(len(mysvar)))]
    svarname1 = ['outu10','outv10','out2mdewtmp','out2mtmp','outconv_ppt','outtot_cld_ice',\
           'outtot_cldwtr','outtot_cld_rain','outvi_div_cld_froz_wtr','outvi_div_cld_liq_wtr','outvi_div_gpot_flux',\
           'outvi_div_ke_flux','outvi_div_mass_flux','outvi_div_moisture_flux','outvi_div_olr_flux','outvi_div_tot_enrgy_flux',\
           'outvi_ke','outvi_pe_inte','outvi_pe_ie_latentenrgy','outvi_temp','outvi_olr','outvi_tot_enrgy','outvi_moisture_div']
   
    svarname = ['u10','v10','2mdewtmp','2mtmp','conv_ppt','tot_cld_ice',\
           'tot_cldwtr','tot_cld_rain','vi_div_cld_froz_wtr','vi_div_cld_liq_wtr','vi_div_gpot_flux',\
           'vi_div_ke_flux','vi_div_mass_flux','vi_div_moisture_flux','vi_div_olr_flux','vi_div_tot_enrgy_flux',\
           'vi_ke','vi_pe_inte','vi_pe_ie_latentenrgy','vi_temp','vi_olr','vi_tot_enrgy','vi_moisture_div']
    svardict = {varnameobj:varobj for (varnameobj,varobj) in zip(svarname,smallsvarout)}
    
    lonselect = dm2.lon.sel(lon=slice(tc_orad[0,:][2],tc_orad[0,:][3])).data
    latselect = np.flipud(dm2.lat.sel(lat=slice(tc_orad[0,:][1],tc_orad[0,:][0])).data)
    lon2d,lat2d = np.meshgrid(lonselect,latselect)

    mymask = createmask(dm=dm2,irad=tc_irad,orad=tc_orad,lonselect=lonselect,latselect=latselect)
    #############################################################################################
    tsdict = {}
    for ind,obj in (enumerate(svarname)):
        tslist = [svardict[svarname[ind]][i,...][~mymask[i]] for i in range(len(mymask))]
        tsdict[svarname1[ind]] = [np.nanmean(obj) for obj in tslist]
    #############################################################################################
    TCs_ts.append(tsdict)
#myvar = [extract_var(var=obj,indices=indices_store) for obj in vars_dm1]

  0%|          | 0/19 [00:00<?, ?it/s]

In [301]:
storeTCdicts = {}
for ind in range(len(TCs_ts)):
    storeTCdicts[stormnames[ind]] = pd.DataFrame.from_dict(TCs_ts[ind])

In [302]:
for ind,obj in enumerate(stormnames):
    storeTCdicts[obj].to_csv(output+'2001_outerslvars_wpac_'+str(obj)+'.csv')

In [303]:
def readyear_automatic(year=None):
    dm2 = xr.open_dataset(datapath+'/slev_vars/radvars_'+str(year)+'.nc')
    #tracklist = sorted(glob.glob('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/besttracks/nio/*_'+str(year)+'*'))
    era5_date = [str(dm2.time[i].data).split('T')[0] for i in range(len(dm2.time))]
    era5_hour = [str(dm2.time[i].data).split('T')[1][0:2] for i in range(len(dm2.time))]
    return dm2,era5_date,era5_hour

dm2,era5_date,era5_hour = readyear_automatic(2001)

In [304]:
###radiation variables

TCr_ts = []
for TCobj in tqdm(stormnames):
    track=tracksDF[tracksDF['name']==TCobj].reset_index()
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mysvar = [extract_var(dataset=dm2,var=obj,indices=indices_store) for obj in (list(dm2.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2
    
    tc_orad=np.empty((len(indices_store),4))
    tc_orad[:,0] = pos[:,0]-8
    tc_orad[:,1] = pos[:,0]+8
    tc_orad[:,2] = pos[:,1]-8
    tc_orad[:,3] = pos[:,1]+8
    ###########################################################################
    smallsvarout = [largearea(dm2,mysvar[i],indices_store) for i in (range(len(mysvar)))]
    
    svarname1 = ['outcape','outinst_10m_wnd_gst','outinst_moisture_flux','outinst_ssh_flux','outsurfmean_swr_flux','outsurfmean_lhf',\
           'outsurfmean_lwr_flux','outsurfmean_shf','outdwnwrdmean_swr_flux','outtopmean_lwr_flux','outtopmean_swr_flux',\
           'outvimean_moisture_div','outsurf_lhf','outsurf_shf','outtot_suprcool_liqwtr','outtot_wtr_vpr']
    
    
    svarname = ['cape','inst_10m_wnd_gst','inst_moisture_flux','inst_ssh_flux','surfmean_swr_flux','surfmean_lhf',\
           'surfmean_lwr_flux','surfmean_shf','dwnwrdmean_swr_flux','topmean_lwr_flux','topmean_swr_flux',\
           'vimean_moisture_div','surf_lhf','surf_shf','tot_suprcool_liqwtr','tot_wtr_vpr']
    
    svardict = {varnameobj:varobj for (varnameobj,varobj) in zip(svarname,smallsvarout)}
    
    lonselect = dm2.lon.sel(lon=slice(tc_orad[0,:][2],tc_orad[0,:][3])).data
    latselect = np.flipud(dm2.lat.sel(lat=slice(tc_orad[0,:][1],tc_orad[0,:][0])).data)
    lon2d,lat2d = np.meshgrid(lonselect,latselect)

    mymask = createmask(dm=dm2,irad=tc_irad,orad=tc_orad,lonselect=lonselect,latselect=latselect)
    #############################################################################################
    tsdict = {}
    for ind,obj in (enumerate(svarname)):
        tslist = [svardict[svarname[ind]][i,...][~mymask[i]] for i in range(len(mymask))]
        tsdict[svarname1[ind]] = [np.nanmean(obj) for obj in tslist]
    #############################################################################################
    TCr_ts.append(tsdict)
#myvar = [extract_var(var=obj,indices=indices_store) for obj in vars_dm1]

  0%|          | 0/19 [00:00<?, ?it/s]

In [305]:
storeTCdicts = {}
for ind in range(len(TCr_ts)):
    storeTCdicts[stormnames[ind]] = pd.DataFrame.from_dict(TCr_ts[ind])

In [306]:
for ind,obj in enumerate(stormnames):
    storeTCdicts[obj].to_csv(output+'2001_outerradvar_wpac_'+str(obj)+'.csv')

In [308]:
def readyear_automatic(year=None):
    dm2 = xr.open_mfdataset([datapath+'/prate/prates_'+str(year)+'.nc',datapath+'/mslp/mslp_'+str(year)+'.nc',datapath+'/sst/sst_'+str(year)+'.nc'])
    #tracklist = sorted(glob.glob('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/besttracks/nio/*_'+str(year)+'*'))
    #era5_date = [str(dm2.time[i].data).split('T')[0] for i in range(len(dm2.time))]
    #era5_hour = [str(dm2.time[i].data).split('T')[1][0:2] for i in range(len(dm2.time))]
    return dm2#,era5_date,era5_hour

dm2 = readyear_automatic(2001)

In [309]:
#########precip, mslp, sst variables############
TCpr_ts = []
for TCobj in tqdm(stormnames):
    track=tracksDF[tracksDF['name']==TCobj].reset_index()
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mysvar = [extract_var(dataset=dm2,var=obj,indices=indices_store) for obj in (list(dm2.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2
    
    tc_orad=np.empty((len(indices_store),4))
    tc_orad[:,0] = pos[:,0]-8
    tc_orad[:,1] = pos[:,0]+8
    tc_orad[:,2] = pos[:,1]-8
    tc_orad[:,3] = pos[:,1]+8
    ###########################################################################
    smallsvarout = [largearea(dm2,mysvar[i],indices_store) for i in (range(len(mysvar)))]
    
    svarname1 = ['outmslp','outconv_rrate','outls_rrate','outmn_conv_prate',\
                 'outmn_ls_prate','outmn_tot_prate','outsst']
    
    svarname = ['mslp','conv_rrate','ls_rrate','mn_conv_prate','mn_ls_prate','mn_tot_prate','sst']
    
    svardict = {varnameobj:varobj for (varnameobj,varobj) in zip(svarname,smallsvarout)}
    
    lonselect = dm2.lon.sel(lon=slice(tc_orad[0,:][2],tc_orad[0,:][3])).data
    latselect = np.flipud(dm2.lat.sel(lat=slice(tc_orad[0,:][1],tc_orad[0,:][0])).data)
    lon2d,lat2d = np.meshgrid(lonselect,latselect)

    mymask = createmask(dm=dm2,irad=tc_irad,orad=tc_orad,lonselect=lonselect,latselect=latselect)
    #############################################################################################
    tsdict = {}
    for ind,obj in (enumerate(svarname)):
        tslist = [svardict[svarname[ind]][i,...][~mymask[i]] for i in range(len(mymask))]
        tsdict[svarname1[ind]] = [np.nanmean(obj) for obj in tslist]
    #############################################################################################
    TCpr_ts.append(tsdict)

  0%|          | 0/19 [00:00<?, ?it/s]

In [310]:
storeTCdicts = {}
for ind in range(len(TCpr_ts)):
    storeTCdicts[stormnames[ind]] = pd.DataFrame.from_dict(TCpr_ts[ind])

In [311]:
for ind,obj in enumerate(stormnames):
    storeTCdicts[obj].to_csv(output+'2001_outpresstvar_wpac_'+str(obj)+'.csv')

In [312]:
dm1=xr.open_mfdataset([datapath+'/vor/vor_2001.nc',datapath+'/rhum/rhum_2001.nc'])

In [313]:
#Pressure vars - vor hum
TCp_ts = []
for TCobj in tqdm(stormnames):
    track=tracksDF[tracksDF['name']==TCobj].reset_index()
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mypvar = [extract_var(dataset=dm1,var=obj,indices=indices_store) for obj in (list(dm1.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2
    
    tc_orad=np.empty((len(indices_store),4))
    tc_orad[:,0] = pos[:,0]-8
    tc_orad[:,1] = pos[:,0]+8
    tc_orad[:,2] = pos[:,1]-8
    tc_orad[:,3] = pos[:,1]+8
    ###########################################################################
    smallpvarout = [largearea_withpres(dm1,mypvar[i],indices_store) for i in (range(len(mypvar)))]    
    pvarname1 = ['outvort','outrhum']
    pvarname = ['vort','rhum']
    pvardict = {varnameobj:varobj for (varnameobj,varobj) in zip(pvarname,smallpvarout)}
    
    lonselect = dm1.lon.sel(lon=slice(tc_orad[0,:][2],tc_orad[0,:][3])).data
    latselect = np.flipud(dm1.lat.sel(lat=slice(tc_orad[0,:][1],tc_orad[0,:][0])).data)
    lon2d,lat2d = np.meshgrid(lonselect,latselect)
    mymask = createmask(dm=dm1,irad=tc_irad,orad=tc_orad,lonselect=lonselect,latselect=latselect)
    #############################################################################################
    ts_pdict = {}
    for ind,obj in (enumerate(pvarname)):
        pvarTS_store = []
        for plevv in range(len(dm1.plev.data)):
            tempvar = pvardict[pvarname[ind]][:,plevv,...]
            tempts = [tempvar[i,...][~mymask[i]] for i in range(len(mymask))]
            tempTSERIES = [np.nanmean(obj) for obj in tempts]
            pvarTS_store.append(tempTSERIES)
        ts_pdict[pvarname1[ind]] = np.asarray(pvarTS_store).transpose()
    #############################################################################################
    TCp_ts.append(ts_pdict)
#myvar = [extract_var(var=obj,indices=indices_store) for obj in vars_dm1]

  0%|          | 0/19 [00:00<?, ?it/s]

In [314]:
dm1=xr.open_dataset(datapath+'/gpot/gpot_2001.nc')

In [315]:
plv = [str(int(obj)) for obj in dm1.plev.data/100]

In [316]:
# Pressure vars - geopotential
TCgp_ts = []
for TCobj in tqdm(stormnames):
    track=tracksDF[tracksDF['name']==TCobj].reset_index()
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mypvar = [extract_var(dataset=dm1,var=obj,indices=indices_store) for obj in (list(dm1.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2
    
    tc_orad=np.empty((len(indices_store),4))
    tc_orad[:,0] = pos[:,0]-8
    tc_orad[:,1] = pos[:,0]+8
    tc_orad[:,2] = pos[:,1]-8
    tc_orad[:,3] = pos[:,1]+8
    ###########################################################################
    smallpvarout = [largearea_withpres(dm1,mypvar[i],indices_store) for i in (range(len(mypvar)))]    
    pvarname1 = ['outgpot']
    pvarname = ['gpot']
    pvardict = {varnameobj:varobj for (varnameobj,varobj) in zip(pvarname,smallpvarout)}
    
    lonselect = dm1.lon.sel(lon=slice(tc_orad[0,:][2],tc_orad[0,:][3])).data
    latselect = np.flipud(dm1.lat.sel(lat=slice(tc_orad[0,:][1],tc_orad[0,:][0])).data)
    lon2d,lat2d = np.meshgrid(lonselect,latselect)
    mymask = createmask(dm=dm1,irad=tc_irad,orad=tc_orad,lonselect=lonselect,latselect=latselect)
    #############################################################################################
    ts_pdict = {}
    for ind,obj in (enumerate(pvarname)):
        pvarTS_store = []
        for plevv in range(len(dm1.plev.data)):
            tempvar = pvardict[pvarname[ind]][:,plevv,...]
            tempts = [tempvar[i,...][~mymask[i]] for i in range(len(mymask))]
            tempTSERIES = [np.nanmean(obj) for obj in tempts]
            pvarTS_store.append(tempTSERIES)
        ts_pdict[pvarname1[ind]] = np.asarray(pvarTS_store).transpose()
    #############################################################################################
    TCgp_ts.append(ts_pdict)
#myvar = [extract_var(var=obj,indices=indices_store) for obj in vars_dm1]

  0%|          | 0/19 [00:00<?, ?it/s]

In [317]:
dm1=xr.open_dataset(datapath+'/tlevs/tlevs_2001.nc')

In [318]:
#Pressure vars - vertical velocity
TCtl_ts = []
for TCobj in tqdm(stormnames):
    track=tracksDF[tracksDF['name']==TCobj].reset_index()
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mypvar = [extract_var(dataset=dm1,var=obj,indices=indices_store) for obj in (list(dm1.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2
    
    tc_orad=np.empty((len(indices_store),4))
    tc_orad[:,0] = pos[:,0]-8
    tc_orad[:,1] = pos[:,0]+8
    tc_orad[:,2] = pos[:,1]-8
    tc_orad[:,3] = pos[:,1]+8
    ###########################################################################
    smallpvarout = [largearea_withpres(dm1,mypvar[i],indices_store) for i in (range(len(mypvar)))]    
    pvarname1 = ['outtemp']
    pvarname = ['temp']
    pvardict = {varnameobj:varobj for (varnameobj,varobj) in zip(pvarname,smallpvarout)}
    
    lonselect = dm1.lon.sel(lon=slice(tc_orad[0,:][2],tc_orad[0,:][3])).data
    latselect = np.flipud(dm1.lat.sel(lat=slice(tc_orad[0,:][1],tc_orad[0,:][0])).data)
    lon2d,lat2d = np.meshgrid(lonselect,latselect)
    mymask = createmask(dm=dm1,irad=tc_irad,orad=tc_orad,lonselect=lonselect,latselect=latselect)
    #############################################################################################
    ts_pdict = {}
    for ind,obj in (enumerate(pvarname)):
        pvarTS_store = []
        for plevv in range(len(dm1.plev.data)):
            tempvar = pvardict[pvarname[ind]][:,plevv,...]
            tempts = [tempvar[i,...][~mymask[i]] for i in range(len(mymask))]
            tempTSERIES = [np.nanmean(obj) for obj in tempts]
            pvarTS_store.append(tempTSERIES)
        ts_pdict[pvarname1[ind]] = np.asarray(pvarTS_store).transpose()
    #############################################################################################
    TCtl_ts.append(ts_pdict)
#myvar = [extract_var(var=obj,indices=indices_store) for obj in vars_dm1]

  0%|          | 0/19 [00:00<?, ?it/s]

In [319]:
dm1=xr.open_dataset(datapath+'/vvel/w_wnd_2001.nc')

In [320]:
#Pressure vars - vertical velocity
TCvv_ts = []
for TCobj in tqdm(stormnames):
    track=tracksDF[tracksDF['name']==TCobj].reset_index()
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mypvar = [extract_var(dataset=dm1,var=obj,indices=indices_store) for obj in (list(dm1.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2
    
    tc_orad=np.empty((len(indices_store),4))
    tc_orad[:,0] = pos[:,0]-8
    tc_orad[:,1] = pos[:,0]+8
    tc_orad[:,2] = pos[:,1]-8
    tc_orad[:,3] = pos[:,1]+8
    ###########################################################################
    smallpvarout = [largearea_withpres(dm1,mypvar[i],indices_store) for i in (range(len(mypvar)))]    
    pvarname1 = ['outvvel']
    pvarname = ['vvel']
    pvardict = {varnameobj:varobj for (varnameobj,varobj) in zip(pvarname,smallpvarout)}
    
    lonselect = dm1.lon.sel(lon=slice(tc_orad[0,:][2],tc_orad[0,:][3])).data
    latselect = np.flipud(dm1.lat.sel(lat=slice(tc_orad[0,:][1],tc_orad[0,:][0])).data)
    lon2d,lat2d = np.meshgrid(lonselect,latselect)
    mymask = createmask(dm=dm1,irad=tc_irad,orad=tc_orad,lonselect=lonselect,latselect=latselect)
    #############################################################################################
    ts_pdict = {}
    for ind,obj in (enumerate(pvarname)):
        pvarTS_store = []
        for plevv in range(len(dm1.plev.data)):
            tempvar = pvardict[pvarname[ind]][:,plevv,...]
            tempts = [tempvar[i,...][~mymask[i]] for i in range(len(mymask))]
            tempTSERIES = [np.nanmean(obj) for obj in tempts]
            pvarTS_store.append(tempTSERIES)
        ts_pdict[pvarname1[ind]] = np.asarray(pvarTS_store).transpose()
    #############################################################################################
    TCvv_ts.append(ts_pdict)
#myvar = [extract_var(var=obj,indices=indices_store) for obj in vars_dm1]

  0%|          | 0/19 [00:00<?, ?it/s]

In [321]:
plv = [str(int(obj)) for obj in dm1.plev.data/100]

In [322]:
dx=xr.open_dataset(datapath+'/tlevs/tlevs_2001.nc')

In [323]:
plev=[str(int(obj)) for obj in dx.plev.data/100]

In [324]:
len(plv),len(plev)

(18, 21)

In [325]:
def create_pandadict(vordict=None,rhumdict=None,gpotdict=None,tempdict=None,vveldict=None,vorlv=plev,rhum=plev,\
                     gpotlv=plev,templv=plev,vvellv=plv):
    tempvordict = {'outvort_'+vorlv[i]:vordict['outvort'][:,i] for i in range(vordict['outvort'].shape[1])}
    temprhumdict = {'outrhum_'+vorlv[i]:rhumdict['outrhum'][:,i] for i in range(rhumdict['outrhum'].shape[1])}
    tempgpotdict = {'outgpot_'+gpotlv[i]:gpotdict['outgpot'][:,i] for i in range(gpotdict['outgpot'].shape[1])}
    temptempdict = {'outtemp_'+templv[i]:tempdict['outtemp'][:,i] for i in range(tempdict['outtemp'].shape[1])}
    tempvveldict = {'outvvel_'+vvellv[i]:vveldict['outvvel'][:,i] for i in range(vveldict['outvvel'].shape[1])}   
    dict1 = {**tempvordict} # change this later
    dict2 = {**temprhumdict,**tempgpotdict}
    dict3 = {**temptempdict,**tempvveldict}
    combine1={**dict1,**dict2} # change this later
    alldict = {**combine1,**dict3} # change this later
    #alldict= {**dict2,**dict3}
    return alldict

In [326]:
storeTCdicts = {}
for ind in range(len(TCp_ts)):
    storeTCdicts[stormnames[ind]] = pd.DataFrame.from_dict(create_pandadict(vordict=TCp_ts[ind],rhumdict=TCp_ts[ind],\
                                                                            gpotdict=TCgp_ts[ind],\
                                                                            tempdict=TCtl_ts[ind],vveldict=TCvv_ts[ind]))

In [327]:
for ind,obj in enumerate(stormnames):
    storeTCdicts[obj].to_csv(output+'2001_outerlevs_wpac_'+str(obj)+'.csv')

In [28]:
slev_kyaar=pd.DataFrame.from_dict(TCs_ts[0])
rlev_kyaar=pd.DataFrame.from_dict(TCr_ts[0])
prlev_kyaar=pd.DataFrame.from_dict(TCpr_ts[0])
prlev_kyaar['outmslp']=prlev_kyaar['outmslp']/100

ts1_kyaar=pd.concat([slev_kyaar,rlev_kyaar,prlev_kyaar], axis=1, join='inner')

tempvortdict = {'outvort_'+plev[i]:TCp_ts[0]['outvort'][:,i] for i in range(21)}
temprhumdict = {'outrhum_'+plev[i]:TCp_ts[0]['outrhum'][:,i] for i in range(21)}
tempgpotdict = {'outgpot_'+plv[i]:TCgp_ts[0]['outgpot'][:,i] for i in range(21)}
tempvveldict = {'outvvel_'+preslv[i]:TCvv_ts[0]['outvvel'][:,i] for i in range(18)}
dict1 = {**tempvortdict,**temprhumdict}
dict0 = {**tempgpotdict,**tempvveldict}

alldict1 = {**dict1,**dict0}

plev_kyaar=pd.DataFrame.from_dict(alldict1)

ts_kyaar=pd.concat([plev_kyaar,ts1_kyaar], axis=1, join='inner')
ts_kyaar


,outvort_10,outvort_20,outvort_30,outvort_50,outvort_70,outvort_100,outvort_150,outvort_200,outvort_250,outvort_300,...,outsurf_shf,outtot_suprcool_liqwtr,outtot_wtr_vpr,outmslp,outconv_rrate,outls_rrate,outmn_conv_prate,outmn_ls_prate,outmn_tot_prate,outsst
0,-0.000013,-0.000014,-0.000004,-8.038129e-07,-0.000004,-0.000006,-0.000013,-0.000008,-0.000007,-0.000008,...,-199661.416667,0.050517,56.292139,1009.195629,1.121959e-04,3.656587e-05,8.370019e-05,3.885394e-05,1.225576e-04,302.105147
1,-0.000013,-0.000014,-0.000005,-1.793650e-06,-0.000004,-0.000007,-0.000015,-0.000011,-0.000008,-0.000009,...,-78138.916667,0.043549,56.497572,1006.490612,6.989970e-05,3.559136e-05,6.958020e-05,3.955193e-05,1.091378e-04,302.131579
2,-0.000014,-0.000012,-0.000007,-7.391286e-07,-0.000005,-0.000007,-0.000016,-0.000012,-0.000007,-0.000007,...,-8880.383333,0.037530,56.420094,1006.097475,3.446986e-05,2.235503e-05,3.583099e-05,2.394101e-05,5.976968e-05,302.146305
3,-0.000013,-0.000009,-0.000007,-9.784087e-07,-0.000004,-0.000009,-0.000014,-0.000011,-0.000007,-0.000006,...,-6168.800000,0.029600,56.349209,1007.823859,2.807571e-05,2.031783e-05,2.774615e-05,1.982677e-05,4.758021e-05,302.157020
4,-0.000012,-0.000008,-0.000006,-2.715919e-06,-0.000002,-0.000009,-0.000012,-0.000011,-0.000007,-0.000005,...,-8927.616667,0.028116,56.841968,1007.708139,3.854219e-05,1.882855e-05,3.682574e-05,2.044520e-05,5.727131e-05,302.184027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,-0.000007,-0.000014,-0.000012,-6.602485e-06,-0.000011,-0.000024,-0.000027,-0.000027,-0.000029,-0.000027,...,21496.816667,0.004420,30.299450,1008.916994,7.229236e-07,2.833704e-07,6.560236e-07,2.926526e-07,9.474034e-07,301.943022
77,-0.000010,-0.000012,-0.000012,-5.209124e-06,-0.000011,-0.000024,-0.000027,-0.000027,-0.000028,-0.000027,...,18061.583333,0.004124,30.371192,1008.305332,1.214258e-06,8.211782e-07,1.200599e-06,9.144967e-07,2.107831e-06,301.939233
78,-0.000012,-0.000011,-0.000011,-4.461858e-06,-0.000011,-0.000024,-0.000029,-0.000028,-0.000028,-0.000027,...,18737.016667,0.005358,29.339513,1008.040143,1.778454e-06,8.129204e-07,1.701464e-06,8.250897e-07,2.526181e-06,302.010758
79,-0.000012,-0.000012,-0.000010,-4.164983e-06,-0.000011,-0.000024,-0.000029,-0.000028,-0.000028,-0.000027,...,10756.916667,0.004095,28.863249,1009.092350,1.622364e-06,5.956118e-07,1.614587e-06,7.673788e-07,2.373507e-06,302.048350


In [39]:
ts_kyaar.to_csv(output+'timeseries_outer_nio_2021.csv')

In [20]:
mainpabuk=pd.read_csv(output+'inner/inner_nio_all_pabuk.csv',delimiter=r",")
mainpabuk.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
mainpabuk=mainpabuk.drop('a', axis=1)

pdf_pabuk=pd.read_csv(target+'2019_pmin_nio_pabuk.csv',delimiter=r",")
pdf_pabuk.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
pdf_pabuk=pdf_pabuk.drop('a', axis=1)

In [21]:
extra=pd.read_csv(output+'nio/outder_2019_nio_pabuk.csv',delimiter=r",")
extra.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
extra=extra.drop('a', axis=1)

In [22]:
slev_pabuk=pd.DataFrame.from_dict(TCs_ts[0])
rlev_pabuk=pd.DataFrame.from_dict(TCr_ts[0])
prlev_pabuk=pd.DataFrame.from_dict(TCpr_ts[0])
prlev_pabuk['outmslp']=prlev_pabuk['outmslp']/100

In [23]:
ts1_pabuk=pd.concat([mainpabuk,extra,slev_pabuk,rlev_pabuk,prlev_pabuk], axis=1, join='inner')

In [24]:
ts1_pabuk

,w10max,div100,div200,div250,div300,div400,div500,eqt1000,eqt200,eqt250,...,outsurf_shf,outtot_suprcool_liqwtr,outtot_wtr_vpr,outmslp,outconv_rrate,outls_rrate,outmn_conv_prate,outmn_ls_prate,outmn_tot_prate,outsst
0,17.673153,0.000032,2.219353e-05,6.673648e-06,2.848649e-06,1.245508e-05,0.000015,335.375427,351.441071,349.637360,...,-141893.083333,0.041908,53.435838,1013.468010,0.000107,0.000074,0.000098,0.000078,0.000175,301.277784
1,17.477364,0.000031,1.403352e-05,5.594714e-06,1.634225e-06,1.353838e-05,0.000013,334.415527,352.598511,349.906494,...,-148228.216667,0.042105,53.321655,1013.453413,0.000108,0.000067,0.000101,0.000071,0.000173,301.318245
2,18.106470,0.000026,1.247141e-05,6.791283e-06,1.284983e-06,1.486074e-05,0.000016,334.079224,353.325348,350.469574,...,-150571.966667,0.047650,53.362784,1010.758319,0.000148,0.000060,0.000155,0.000059,0.000213,301.345952
3,17.562447,0.000027,2.166161e-05,4.743541e-07,-1.978659e-06,1.404193e-05,0.000014,334.612854,353.475494,350.679260,...,-42168.950000,0.043336,53.360059,1010.112013,0.000075,0.000061,0.000092,0.000062,0.000154,301.354430
4,17.026632,0.000026,1.122398e-05,1.776409e-05,5.837156e-06,2.259514e-06,0.000012,335.253510,352.463074,350.376038,...,-9152.327801,0.037071,54.266717,1011.877284,0.000081,0.000060,0.000083,0.000060,0.000143,301.367659
5,17.883175,0.000026,1.412555e-05,1.466526e-05,7.506639e-06,3.567956e-06,0.000017,335.245911,352.335297,350.027557,...,-14229.166667,0.049405,54.499174,1012.768839,0.000075,0.000059,0.000070,0.000056,0.000125,301.376774
6,17.705912,0.000033,1.698328e-05,4.081766e-06,-2.125693e-06,6.212415e-06,0.000029,335.514618,352.070374,349.906250,...,-22251.883333,0.051768,54.016899,1011.741212,0.000077,0.000076,0.000074,0.000066,0.000141,301.383425
7,18.306595,0.000032,2.037618e-05,1.821254e-06,-5.462623e-06,1.178157e-05,0.000021,332.922424,351.774384,349.920959,...,-25729.716667,0.058166,53.144870,1011.111317,0.000097,0.000095,0.000098,0.000090,0.000187,301.399550
8,15.792764,0.000027,1.457374e-05,2.087121e-06,-4.151153e-06,-2.182498e-06,0.000012,330.039001,352.255432,350.169556,...,-34930.766667,0.060602,52.329786,1012.507376,0.000081,0.000098,0.000086,0.000096,0.000182,301.378313
9,16.368551,0.000029,6.476554e-06,-8.699307e-06,-1.521692e-05,1.289164e-06,0.000016,332.401062,351.971497,349.480194,...,-133059.850000,0.059614,52.640927,1013.599438,0.000087,0.000095,0.000080,0.000095,0.000174,301.368760


In [29]:
tempvortdict = {'outvort_'+plev[i]:TCp_ts[0]['outvort'][:,i] for i in range(21)}
temprhumdict = {'outrhum_'+plev[i]:TCp_ts[0]['outrhum'][:,i] for i in range(21)}
tempgpotdict = {'outgpot_'+plv[i]:TCgp_ts[0]['outgpot'][:,i] for i in range(20)}
tempvveldict = {'outvvel_'+preslv[i]:TCvv_ts[0]['outvvel'][:,i] for i in range(18)}
dict1 = {**tempvortdict,**temprhumdict}
dict3 = {**tempgpotdict,**tempvveldict}

alldict1 = {**dict1,**dict3}

In [30]:
plev_pabuk=pd.DataFrame.from_dict(alldict1)

In [31]:
ts_pabuk=pd.concat([pdf_pabuk,ts1_pabuk,plev_pabuk], axis=1, join='inner')
ts_pabuk

,Pmin,w10max,div100,div200,div250,div300,div400,div500,eqt1000,eqt200,...,outvvel_250,outvvel_300,outvvel_400,outvvel_500,outvvel_600,outvvel_700,outvvel_850,outvvel_925,outvvel_975,outvvel_1000
0,1006.001875,17.673153,0.000032,2.219353e-05,6.673648e-06,2.848649e-06,1.245508e-05,0.000015,335.375427,351.441071,...,-0.134500,-0.145284,-0.117964,-0.089915,-0.089689,-0.079635,-0.058293,-0.042522,-0.026545,-0.017776
1,1006.361875,17.477364,0.000031,1.403352e-05,5.594714e-06,1.634225e-06,1.353838e-05,0.000013,334.415527,352.598511,...,-0.134701,-0.146680,-0.120218,-0.089581,-0.084764,-0.078276,-0.062532,-0.039610,-0.023329,-0.016027
2,1003.893750,18.106470,0.000026,1.247141e-05,6.791283e-06,1.284983e-06,1.486074e-05,0.000016,334.079224,353.325348,...,-0.154768,-0.165026,-0.134061,-0.107039,-0.098483,-0.090815,-0.079946,-0.058220,-0.039631,-0.033148
3,1002.906875,17.562447,0.000027,2.166161e-05,4.743541e-07,-1.978659e-06,1.404193e-05,0.000014,334.612854,353.475494,...,-0.140605,-0.146367,-0.111128,-0.081864,-0.068382,-0.056991,-0.043414,-0.019018,0.003211,0.010729
4,1004.182500,17.026632,0.000026,1.122398e-05,1.776409e-05,5.837156e-06,2.259514e-06,0.000012,335.253510,352.463074,...,-0.135550,-0.139399,-0.103081,-0.077586,-0.072925,-0.071134,-0.053590,-0.019854,0.007859,0.018991
5,1004.449375,17.883175,0.000026,1.412555e-05,1.466526e-05,7.506639e-06,3.567956e-06,0.000017,335.245911,352.335297,...,-0.126358,-0.133883,-0.102536,-0.077375,-0.074852,-0.073559,-0.061708,-0.033066,-0.007617,0.002563
6,1003.071875,17.705912,0.000033,1.698328e-05,4.081766e-06,-2.125693e-06,6.212415e-06,0.000029,335.514618,352.070374,...,-0.130540,-0.144989,-0.124502,-0.097040,-0.089466,-0.084891,-0.068174,-0.039197,-0.015825,-0.007000
7,1001.766250,18.306595,0.000032,2.037618e-05,1.821254e-06,-5.462623e-06,1.178157e-05,0.000021,332.922424,351.774384,...,-0.118974,-0.136516,-0.122984,-0.097059,-0.083057,-0.072506,-0.044576,-0.012355,0.009516,0.017880
8,1003.541250,15.792764,0.000027,1.457374e-05,2.087121e-06,-4.151153e-06,-2.182498e-06,0.000012,330.039001,352.255432,...,-0.115978,-0.136877,-0.137872,-0.112906,-0.096078,-0.084602,-0.040399,-0.008360,0.015667,0.026039
9,1004.440000,16.368551,0.000029,6.476554e-06,-8.699307e-06,-1.521692e-05,1.289164e-06,0.000016,332.401062,351.971497,...,-0.129164,-0.147550,-0.147436,-0.125392,-0.115892,-0.102236,-0.061502,-0.031335,-0.014853,-0.008181


In [32]:
df1=ts_pabuk
df1=df1.drop('ws10', axis=1)
df1=df1.drop('u10', axis=1)
df1=df1.drop('v10', axis=1)
df1=df1.drop('out_ws10', axis=1)
df1=df1.drop('outu10', axis=1)
df1=df1.drop('outv10', axis=1)
df1=df1.drop('vi_div_ke_flux', axis=1)
df1=df1.drop('vi_div_gpot_flux', axis=1)
df1=df1.drop('vi_ke', axis=1)
df1=df1.drop('vi_div_olr_flux', axis=1)
df1=df1.drop('vi_temp', axis=1)
df1=df1.drop('inst_10m_wnd_gst', axis=1)
df1=df1.drop('surfmean_swr_flux', axis=1)
df1=df1.drop('surfmean_lwr_flux', axis=1)
df1=df1.drop('dwnwrdmean_swr_flux', axis=1)
df1=df1.drop('topmean_lwr_flux', axis=1)
df1=df1.drop('topmean_swr_flux', axis=1)
df1=df1.drop('outvi_div_ke_flux', axis=1)
df1=df1.drop('outvi_div_gpot_flux', axis=1)
df1=df1.drop('outvi_ke', axis=1)
df1=df1.drop('outvi_div_olr_flux', axis=1)
df1=df1.drop('outvi_temp', axis=1)
df1=df1.drop('outinst_10m_wnd_gst', axis=1)
df1=df1.drop('outsurfmean_swr_flux', axis=1)
df1=df1.drop('outsurfmean_lwr_flux', axis=1)
df1=df1.drop('outdwnwrdmean_swr_flux', axis=1)
df1=df1.drop('outtopmean_lwr_flux', axis=1)
df1=df1.drop('outtopmean_swr_flux', axis=1)
df1=df1.drop('outmslp', axis=1)
df1=df1.drop('mslp', axis=1)
df1

,Pmin,w10max,div100,div200,div250,div300,div400,div500,eqt1000,eqt200,...,outvvel_250,outvvel_300,outvvel_400,outvvel_500,outvvel_600,outvvel_700,outvvel_850,outvvel_925,outvvel_975,outvvel_1000
0,1006.001875,17.673153,0.000032,2.219353e-05,6.673648e-06,2.848649e-06,1.245508e-05,0.000015,335.375427,351.441071,...,-0.134500,-0.145284,-0.117964,-0.089915,-0.089689,-0.079635,-0.058293,-0.042522,-0.026545,-0.017776
1,1006.361875,17.477364,0.000031,1.403352e-05,5.594714e-06,1.634225e-06,1.353838e-05,0.000013,334.415527,352.598511,...,-0.134701,-0.146680,-0.120218,-0.089581,-0.084764,-0.078276,-0.062532,-0.039610,-0.023329,-0.016027
2,1003.893750,18.106470,0.000026,1.247141e-05,6.791283e-06,1.284983e-06,1.486074e-05,0.000016,334.079224,353.325348,...,-0.154768,-0.165026,-0.134061,-0.107039,-0.098483,-0.090815,-0.079946,-0.058220,-0.039631,-0.033148
3,1002.906875,17.562447,0.000027,2.166161e-05,4.743541e-07,-1.978659e-06,1.404193e-05,0.000014,334.612854,353.475494,...,-0.140605,-0.146367,-0.111128,-0.081864,-0.068382,-0.056991,-0.043414,-0.019018,0.003211,0.010729
4,1004.182500,17.026632,0.000026,1.122398e-05,1.776409e-05,5.837156e-06,2.259514e-06,0.000012,335.253510,352.463074,...,-0.135550,-0.139399,-0.103081,-0.077586,-0.072925,-0.071134,-0.053590,-0.019854,0.007859,0.018991
5,1004.449375,17.883175,0.000026,1.412555e-05,1.466526e-05,7.506639e-06,3.567956e-06,0.000017,335.245911,352.335297,...,-0.126358,-0.133883,-0.102536,-0.077375,-0.074852,-0.073559,-0.061708,-0.033066,-0.007617,0.002563
6,1003.071875,17.705912,0.000033,1.698328e-05,4.081766e-06,-2.125693e-06,6.212415e-06,0.000029,335.514618,352.070374,...,-0.130540,-0.144989,-0.124502,-0.097040,-0.089466,-0.084891,-0.068174,-0.039197,-0.015825,-0.007000
7,1001.766250,18.306595,0.000032,2.037618e-05,1.821254e-06,-5.462623e-06,1.178157e-05,0.000021,332.922424,351.774384,...,-0.118974,-0.136516,-0.122984,-0.097059,-0.083057,-0.072506,-0.044576,-0.012355,0.009516,0.017880
8,1003.541250,15.792764,0.000027,1.457374e-05,2.087121e-06,-4.151153e-06,-2.182498e-06,0.000012,330.039001,352.255432,...,-0.115978,-0.136877,-0.137872,-0.112906,-0.096078,-0.084602,-0.040399,-0.008360,0.015667,0.026039
9,1004.440000,16.368551,0.000029,6.476554e-06,-8.699307e-06,-1.521692e-05,1.289164e-06,0.000016,332.401062,351.971497,...,-0.129164,-0.147550,-0.147436,-0.125392,-0.115892,-0.102236,-0.061502,-0.031335,-0.014853,-0.008181


In [33]:
df1.to_csv(output+'timeseries_all_nio_pabuk.csv')


In [39]:
df1.to_csv(output+'timeseries_all_nio_vayu.csv')

In [40]:
#storms 4

mainkyaar=pd.read_csv(output+'inner/inner_nio_20210924.csv',delimiter=r",")
mainkyaar.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
mainkyaar=mainkyaar.drop('a', axis=1)

pdf_kyaar=pd.read_csv(target+'2019_pmin_nio_kyaar.csv',delimiter=r",")
pdf_kyaar.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
pdf_kyaar=pdf_kyaar.drop('a', axis=1)

extra=pd.read_csv(output+'nio/outder_2019_nio_kyaar.csv',delimiter=r",")
extra.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
extra=extra.drop('a', axis=1)


slev_kyaar=pd.DataFrame.from_dict(TCs_ts[3])
rlev_kyaar=pd.DataFrame.from_dict(TCr_ts[3])
prlev_kyaar=pd.DataFrame.from_dict(TCpr_ts[3])
prlev_kyaar['outmslp']=prlev_kyaar['outmslp']/100

ts1_kyaar=pd.concat([mainkyaar,extra,slev_kyaar,rlev_kyaar,prlev_kyaar], axis=1, join='inner')

tempvortdict = {'outvort_'+plev[i]:TCp_ts[3]['outvort'][:,i] for i in range(21)}
temprhumdict = {'outrhum_'+plev[i]:TCp_ts[3]['outrhum'][:,i] for i in range(21)}
tempgpotdict = {'outgpot_'+plev[i]:TCgp_ts[3]['outgpot'][:,i] for i in range(21)}
tempvveldict = {'outvvel_'+preslv[i]:TCvv_ts[3]['outvvel'][:,i] for i in range(18)}
dict1 = {**tempvortdict,**temprhumdict}
dict3 = {**tempgpotdict,**tempvveldict}

alldict1 = {**dict1,**dict3}

plev_kyaar=pd.DataFrame.from_dict(alldict1)

ts_kyaar=pd.concat([pdf_kyaar,ts1_kyaar,plev_kyaar], axis=1, join='inner')
ts_kyaar


,Pmin,w10max,div100,div200,div250,div300,div400,div500,eqt1000,eqt200,...,outvvel_250,outvvel_300,outvvel_400,outvvel_500,outvvel_600,outvvel_700,outvvel_850,outvvel_925,outvvel_975,outvvel_1000
0,1005.583750,12.210130,3.294097e-05,0.000011,-4.351196e-06,-8.201038e-06,-1.458761e-05,-0.000004,335.664368,351.300537,...,-0.088679,-0.113841,-0.099230,-0.084361,-0.071985,-0.053667,-0.027510,-0.020349,-0.014500,-0.010503
1,1003.560625,12.457776,3.885604e-05,0.000019,4.416178e-06,-5.564861e-06,-6.433176e-06,0.000005,335.707062,350.870270,...,-0.090466,-0.107301,-0.088883,-0.066050,-0.060383,-0.053784,-0.028668,-0.022579,-0.015735,-0.010568
2,1002.400000,12.454913,4.707007e-05,0.000035,2.491784e-06,-1.312214e-05,-4.629718e-06,-0.000006,335.804138,351.189575,...,-0.064932,-0.095481,-0.095538,-0.065001,-0.040398,-0.022053,-0.004843,-0.002101,0.003398,0.009405
3,1003.980000,13.925591,1.859126e-05,0.000031,8.427362e-06,1.655502e-06,-4.255817e-06,-0.000008,335.457703,350.354797,...,-0.085823,-0.104995,-0.083314,-0.074922,-0.071497,-0.065442,-0.023199,-0.004677,0.010564,0.017379
4,1004.383125,14.422464,1.300272e-05,0.000015,1.787337e-05,1.425643e-05,-3.775083e-06,-0.000002,335.413269,351.007812,...,-0.096154,-0.105856,-0.086384,-0.080117,-0.080251,-0.076402,-0.052181,-0.039073,-0.028814,-0.023920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,1008.973125,11.359031,1.127456e-05,0.000008,-6.519937e-07,-5.312598e-07,2.857952e-06,0.000002,332.913239,349.144104,...,-0.015085,0.003149,0.022370,0.033467,0.030259,0.014582,-0.034112,-0.018542,-0.006842,-0.003905
89,1007.999375,11.423564,1.215437e-05,0.000001,-3.047489e-08,-4.133225e-06,1.100732e-06,0.000004,332.137360,349.189850,...,-0.013655,-0.000114,0.014238,0.016360,0.011420,-0.006509,-0.038913,-0.018567,-0.006109,-0.003168
90,1007.996250,11.131371,2.257598e-05,0.000003,1.788371e-06,3.810991e-06,9.428027e-08,0.000003,332.140198,349.692963,...,-0.005288,0.007914,0.024641,0.025573,0.021973,0.013227,-0.014235,0.006961,0.018196,0.021634
91,1009.318750,10.158909,8.067890e-06,0.000006,7.562204e-06,3.510273e-06,-7.782533e-06,0.000003,333.383331,349.348724,...,-0.000050,0.007840,0.010524,-0.001270,-0.017049,-0.017233,-0.020905,0.012821,0.028990,0.035913


In [41]:
df1=ts_kyaar
df1=df1.drop('ws10', axis=1)
df1=df1.drop('u10', axis=1)
df1=df1.drop('v10', axis=1)
df1=df1.drop('out_ws10', axis=1)
df1=df1.drop('outu10', axis=1)
df1=df1.drop('outv10', axis=1)
df1=df1.drop('vi_div_ke_flux', axis=1)
df1=df1.drop('vi_div_gpot_flux', axis=1)
df1=df1.drop('vi_ke', axis=1)
df1=df1.drop('vi_div_olr_flux', axis=1)
df1=df1.drop('vi_temp', axis=1)
df1=df1.drop('inst_10m_wnd_gst', axis=1)
df1=df1.drop('surfmean_swr_flux', axis=1)
df1=df1.drop('surfmean_lwr_flux', axis=1)
df1=df1.drop('dwnwrdmean_swr_flux', axis=1)
df1=df1.drop('topmean_lwr_flux', axis=1)
df1=df1.drop('topmean_swr_flux', axis=1)
df1=df1.drop('outvi_div_ke_flux', axis=1)
df1=df1.drop('outvi_div_gpot_flux', axis=1)
df1=df1.drop('outvi_ke', axis=1)
df1=df1.drop('outvi_div_olr_flux', axis=1)
df1=df1.drop('outvi_temp', axis=1)
df1=df1.drop('outinst_10m_wnd_gst', axis=1)
df1=df1.drop('outsurfmean_swr_flux', axis=1)
df1=df1.drop('outsurfmean_lwr_flux', axis=1)
df1=df1.drop('outdwnwrdmean_swr_flux', axis=1)
df1=df1.drop('outtopmean_lwr_flux', axis=1)
df1=df1.drop('outtopmean_swr_flux', axis=1)
df1=df1.drop('outmslp', axis=1)
df1=df1.drop('mslp', axis=1)
df1

,Pmin,w10max,div100,div200,div250,div300,div400,div500,eqt1000,eqt200,...,outvvel_250,outvvel_300,outvvel_400,outvvel_500,outvvel_600,outvvel_700,outvvel_850,outvvel_925,outvvel_975,outvvel_1000
0,1005.583750,12.210130,3.294097e-05,0.000011,-4.351196e-06,-8.201038e-06,-1.458761e-05,-0.000004,335.664368,351.300537,...,-0.088679,-0.113841,-0.099230,-0.084361,-0.071985,-0.053667,-0.027510,-0.020349,-0.014500,-0.010503
1,1003.560625,12.457776,3.885604e-05,0.000019,4.416178e-06,-5.564861e-06,-6.433176e-06,0.000005,335.707062,350.870270,...,-0.090466,-0.107301,-0.088883,-0.066050,-0.060383,-0.053784,-0.028668,-0.022579,-0.015735,-0.010568
2,1002.400000,12.454913,4.707007e-05,0.000035,2.491784e-06,-1.312214e-05,-4.629718e-06,-0.000006,335.804138,351.189575,...,-0.064932,-0.095481,-0.095538,-0.065001,-0.040398,-0.022053,-0.004843,-0.002101,0.003398,0.009405
3,1003.980000,13.925591,1.859126e-05,0.000031,8.427362e-06,1.655502e-06,-4.255817e-06,-0.000008,335.457703,350.354797,...,-0.085823,-0.104995,-0.083314,-0.074922,-0.071497,-0.065442,-0.023199,-0.004677,0.010564,0.017379
4,1004.383125,14.422464,1.300272e-05,0.000015,1.787337e-05,1.425643e-05,-3.775083e-06,-0.000002,335.413269,351.007812,...,-0.096154,-0.105856,-0.086384,-0.080117,-0.080251,-0.076402,-0.052181,-0.039073,-0.028814,-0.023920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,1008.973125,11.359031,1.127456e-05,0.000008,-6.519937e-07,-5.312598e-07,2.857952e-06,0.000002,332.913239,349.144104,...,-0.015085,0.003149,0.022370,0.033467,0.030259,0.014582,-0.034112,-0.018542,-0.006842,-0.003905
89,1007.999375,11.423564,1.215437e-05,0.000001,-3.047489e-08,-4.133225e-06,1.100732e-06,0.000004,332.137360,349.189850,...,-0.013655,-0.000114,0.014238,0.016360,0.011420,-0.006509,-0.038913,-0.018567,-0.006109,-0.003168
90,1007.996250,11.131371,2.257598e-05,0.000003,1.788371e-06,3.810991e-06,9.428027e-08,0.000003,332.140198,349.692963,...,-0.005288,0.007914,0.024641,0.025573,0.021973,0.013227,-0.014235,0.006961,0.018196,0.021634
91,1009.318750,10.158909,8.067890e-06,0.000006,7.562204e-06,3.510273e-06,-7.782533e-06,0.000003,333.383331,349.348724,...,-0.000050,0.007840,0.010524,-0.001270,-0.017049,-0.017233,-0.020905,0.012821,0.028990,0.035913


In [42]:
df1.to_csv(output+'timeseries_all_nio_kyaar.csv')

In [43]:
#storms 5

mainmaha=pd.read_csv(output+'inner/inner_nio_all_maha.csv',delimiter=r",")
mainmaha.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
mainmaha=mainmaha.drop('a', axis=1)

pdf_maha=pd.read_csv(target+'2019_pmin_nio_maha.csv',delimiter=r",")
pdf_maha.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
pdf_maha=pdf_maha.drop('a', axis=1)

extra=pd.read_csv(output+'nio/outder_2019_nio_maha.csv',delimiter=r",")
extra.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
extra=extra.drop('a', axis=1)


slev_maha=pd.DataFrame.from_dict(TCs_ts[4])
rlev_maha=pd.DataFrame.from_dict(TCr_ts[4])
prlev_maha=pd.DataFrame.from_dict(TCpr_ts[4])
prlev_maha['outmslp']=prlev_maha['outmslp']/100

ts1_maha=pd.concat([mainmaha,extra,slev_maha,rlev_maha,prlev_maha], axis=1, join='inner')

tempvortdict = {'outvort_'+plev[i]:TCp_ts[4]['outvort'][:,i] for i in range(21)}
temprhumdict = {'outrhum_'+plev[i]:TCp_ts[4]['outrhum'][:,i] for i in range(21)}
tempgpotdict = {'outgpot_'+plev[i]:TCgp_ts[4]['outgpot'][:,i] for i in range(21)}
tempvveldict = {'outvvel_'+preslv[i]:TCvv_ts[4]['outvvel'][:,i] for i in range(18)}
dict1 = {**tempvortdict,**temprhumdict}
dict3 = {**tempgpotdict,**tempvveldict}

alldict1 = {**dict1,**dict3}

plev_maha=pd.DataFrame.from_dict(alldict1)

ts_maha=pd.concat([pdf_maha,ts1_maha,plev_maha], axis=1, join='inner')
ts_maha


,Pmin,w10max,div100,div200,div250,div300,div400,div500,eqt1000,eqt200,...,outvvel_250,outvvel_300,outvvel_400,outvvel_500,outvvel_600,outvvel_700,outvvel_850,outvvel_925,outvvel_975,outvvel_1000
0,1008.783125,10.584517,2.837010e-05,0.000024,1.950195e-05,0.000004,-1.604122e-05,-1.840813e-05,332.681274,348.209381,...,-0.060362,-0.077363,-0.078413,-0.063772,-0.058543,-0.065132,-0.060809,-0.047973,-0.035513,-0.029360
1,1006.049375,10.558450,2.121680e-05,0.000022,1.964651e-05,-0.000001,-1.652125e-05,-1.184978e-05,332.985962,348.588318,...,-0.060141,-0.081287,-0.082261,-0.063592,-0.055546,-0.057973,-0.051523,-0.037779,-0.024293,-0.016833
2,1006.037500,11.040154,6.358685e-06,0.000013,1.192380e-05,-0.000001,-1.260422e-05,-5.331170e-07,328.593994,349.121735,...,-0.050589,-0.068221,-0.052440,-0.024127,-0.005297,-0.010812,-0.024750,-0.012793,0.005072,0.014823
3,1008.734375,10.997210,1.729269e-06,0.000016,1.408634e-05,-0.000006,-1.870668e-05,2.570313e-06,322.613495,348.714417,...,-0.051050,-0.065846,-0.057018,-0.036197,-0.013954,-0.016640,-0.029719,-0.015629,0.004684,0.015498
4,1008.497500,10.445864,1.016564e-06,0.000012,1.063742e-05,-0.000002,-1.007314e-05,6.593957e-06,325.106506,348.548950,...,-0.061838,-0.076014,-0.068527,-0.062537,-0.052894,-0.057948,-0.064867,-0.045585,-0.023142,-0.011746
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,1009.275000,7.144583,-2.414607e-06,0.000003,1.236842e-06,0.000007,1.120639e-06,-2.032241e-07,313.049408,347.872192,...,0.007436,0.002987,-0.013309,-0.011176,0.003273,0.013195,0.026800,0.025481,0.009196,-0.001885
107,1011.270625,7.692482,-1.646526e-06,0.000009,-3.231819e-07,-0.000003,1.121476e-06,8.999232e-07,312.210541,347.748535,...,0.011738,0.007397,-0.003070,0.002988,0.017074,0.030592,0.039098,0.039259,0.029951,0.021451
108,1011.425625,8.196002,2.357962e-07,0.000010,-3.733933e-06,-0.000007,-5.031806e-07,1.649358e-06,311.796631,347.566376,...,0.000833,-0.006535,-0.018069,-0.013248,-0.008441,-0.004949,0.008362,0.014262,0.011275,0.007224
109,1009.437500,7.659556,1.868870e-06,0.000010,-6.726149e-06,-0.000005,-1.689582e-06,3.722544e-06,312.833069,347.423798,...,-0.012944,-0.016545,-0.025843,-0.020772,-0.020076,-0.020066,-0.011609,-0.002039,-0.000054,-0.000797


In [44]:
df1=ts_maha
df1=df1.drop('ws10', axis=1)
df1=df1.drop('u10', axis=1)
df1=df1.drop('v10', axis=1)
df1=df1.drop('out_ws10', axis=1)
df1=df1.drop('outu10', axis=1)
df1=df1.drop('outv10', axis=1)
df1=df1.drop('vi_div_ke_flux', axis=1)
df1=df1.drop('vi_div_gpot_flux', axis=1)
df1=df1.drop('vi_ke', axis=1)
df1=df1.drop('vi_div_olr_flux', axis=1)
df1=df1.drop('vi_temp', axis=1)
df1=df1.drop('inst_10m_wnd_gst', axis=1)
df1=df1.drop('surfmean_swr_flux', axis=1)
df1=df1.drop('surfmean_lwr_flux', axis=1)
df1=df1.drop('dwnwrdmean_swr_flux', axis=1)
df1=df1.drop('topmean_lwr_flux', axis=1)
df1=df1.drop('topmean_swr_flux', axis=1)
df1=df1.drop('outvi_div_ke_flux', axis=1)
df1=df1.drop('outvi_div_gpot_flux', axis=1)
df1=df1.drop('outvi_ke', axis=1)
df1=df1.drop('outvi_div_olr_flux', axis=1)
df1=df1.drop('outvi_temp', axis=1)
df1=df1.drop('outinst_10m_wnd_gst', axis=1)
df1=df1.drop('outsurfmean_swr_flux', axis=1)
df1=df1.drop('outsurfmean_lwr_flux', axis=1)
df1=df1.drop('outdwnwrdmean_swr_flux', axis=1)
df1=df1.drop('outtopmean_lwr_flux', axis=1)
df1=df1.drop('outtopmean_swr_flux', axis=1)
df1=df1.drop('outmslp', axis=1)
df1=df1.drop('mslp', axis=1)
df1

,Pmin,w10max,div100,div200,div250,div300,div400,div500,eqt1000,eqt200,...,outvvel_250,outvvel_300,outvvel_400,outvvel_500,outvvel_600,outvvel_700,outvvel_850,outvvel_925,outvvel_975,outvvel_1000
0,1008.783125,10.584517,2.837010e-05,0.000024,1.950195e-05,0.000004,-1.604122e-05,-1.840813e-05,332.681274,348.209381,...,-0.060362,-0.077363,-0.078413,-0.063772,-0.058543,-0.065132,-0.060809,-0.047973,-0.035513,-0.029360
1,1006.049375,10.558450,2.121680e-05,0.000022,1.964651e-05,-0.000001,-1.652125e-05,-1.184978e-05,332.985962,348.588318,...,-0.060141,-0.081287,-0.082261,-0.063592,-0.055546,-0.057973,-0.051523,-0.037779,-0.024293,-0.016833
2,1006.037500,11.040154,6.358685e-06,0.000013,1.192380e-05,-0.000001,-1.260422e-05,-5.331170e-07,328.593994,349.121735,...,-0.050589,-0.068221,-0.052440,-0.024127,-0.005297,-0.010812,-0.024750,-0.012793,0.005072,0.014823
3,1008.734375,10.997210,1.729269e-06,0.000016,1.408634e-05,-0.000006,-1.870668e-05,2.570313e-06,322.613495,348.714417,...,-0.051050,-0.065846,-0.057018,-0.036197,-0.013954,-0.016640,-0.029719,-0.015629,0.004684,0.015498
4,1008.497500,10.445864,1.016564e-06,0.000012,1.063742e-05,-0.000002,-1.007314e-05,6.593957e-06,325.106506,348.548950,...,-0.061838,-0.076014,-0.068527,-0.062537,-0.052894,-0.057948,-0.064867,-0.045585,-0.023142,-0.011746
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,1009.275000,7.144583,-2.414607e-06,0.000003,1.236842e-06,0.000007,1.120639e-06,-2.032241e-07,313.049408,347.872192,...,0.007436,0.002987,-0.013309,-0.011176,0.003273,0.013195,0.026800,0.025481,0.009196,-0.001885
107,1011.270625,7.692482,-1.646526e-06,0.000009,-3.231819e-07,-0.000003,1.121476e-06,8.999232e-07,312.210541,347.748535,...,0.011738,0.007397,-0.003070,0.002988,0.017074,0.030592,0.039098,0.039259,0.029951,0.021451
108,1011.425625,8.196002,2.357962e-07,0.000010,-3.733933e-06,-0.000007,-5.031806e-07,1.649358e-06,311.796631,347.566376,...,0.000833,-0.006535,-0.018069,-0.013248,-0.008441,-0.004949,0.008362,0.014262,0.011275,0.007224
109,1009.437500,7.659556,1.868870e-06,0.000010,-6.726149e-06,-0.000005,-1.689582e-06,3.722544e-06,312.833069,347.423798,...,-0.012944,-0.016545,-0.025843,-0.020772,-0.020076,-0.020066,-0.011609,-0.002039,-0.000054,-0.000797


In [45]:
df1.to_csv(output+'timeseries_all_nio_maha.csv')

In [46]:
#storms 6

mainbulbul=pd.read_csv(output+'inner/inner_nio_all_bulbul.csv',delimiter=r",")
mainbulbul.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
mainbulbul=mainbulbul.drop('a', axis=1)

pdf_bulbul=pd.read_csv(target+'2019_pmin_nio_bulbul.csv',delimiter=r",")
pdf_bulbul.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
pdf_bulbul=pdf_bulbul.drop('a', axis=1)

extra=pd.read_csv(output+'nio/outder_2019_nio_bulbul.csv',delimiter=r",")
extra.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
extra=extra.drop('a', axis=1)


slev_bulbul=pd.DataFrame.from_dict(TCs_ts[5])
rlev_bulbul=pd.DataFrame.from_dict(TCr_ts[5])
prlev_bulbul=pd.DataFrame.from_dict(TCpr_ts[5])
prlev_bulbul['outmslp']=prlev_bulbul['outmslp']/100

ts1_bulbul=pd.concat([mainbulbul,extra,slev_bulbul,rlev_bulbul,prlev_bulbul], axis=1, join='inner')

tempvortdict = {'outvort_'+plev[i]:TCp_ts[5]['outvort'][:,i] for i in range(21)}
temprhumdict = {'outrhum_'+plev[i]:TCp_ts[5]['outrhum'][:,i] for i in range(21)}
tempgpotdict = {'outgpot_'+plev[i]:TCgp_ts[5]['outgpot'][:,i] for i in range(21)}
tempvveldict = {'outvvel_'+preslv[i]:TCvv_ts[5]['outvvel'][:,i] for i in range(18)}
dict1 = {**tempvortdict,**temprhumdict}
dict3 = {**tempgpotdict,**tempvveldict}

alldict1 = {**dict1,**dict3}

plev_bulbul=pd.DataFrame.from_dict(alldict1)

ts_bulbul=pd.concat([pdf_bulbul,ts1_bulbul,plev_bulbul], axis=1, join='inner')
ts_bulbul


,Pmin,w10max,div100,div200,div250,div300,div400,div500,eqt1000,eqt200,...,outvvel_250,outvvel_300,outvvel_400,outvvel_500,outvvel_600,outvvel_700,outvvel_850,outvvel_925,outvvel_975,outvvel_1000
0,1009.906250,10.584517,0.000004,1.360982e-05,4.113630e-06,-3.951576e-06,-1.947213e-06,-5.324125e-06,332.753601,349.601562,...,-6.098684e-03,-0.014341,-0.008909,-0.010385,-0.015297,-0.024262,-0.022208,0.000484,0.022405,0.031502
1,1011.211250,10.558450,-0.000007,1.180217e-05,5.048701e-06,7.306909e-07,2.372099e-07,7.937452e-07,328.466949,349.497437,...,-1.208216e-02,-0.019809,-0.015735,-0.027773,-0.030505,-0.037675,-0.034355,-0.017912,-0.008622,-0.005007
2,1009.446875,11.040154,-0.000008,1.765924e-05,1.021891e-05,4.357842e-06,1.986422e-06,1.384878e-06,326.333008,349.679840,...,-1.101356e-02,-0.023957,-0.037064,-0.062312,-0.065495,-0.066358,-0.055082,-0.043915,-0.040607,-0.041222
3,1007.248750,10.997210,0.000002,1.242364e-05,-3.431449e-06,-7.650060e-06,9.642074e-07,4.334469e-06,327.476135,350.137573,...,-4.615784e-03,-0.020518,-0.038315,-0.062682,-0.062309,-0.057834,-0.035955,-0.014446,-0.005745,-0.003362
4,1008.545000,10.445864,0.000016,1.031823e-05,3.785833e-06,-3.954945e-06,2.207007e-06,6.988461e-06,330.158203,350.548615,...,-1.248678e-03,-0.013755,-0.025165,-0.045954,-0.049125,-0.046761,-0.021107,0.007155,0.024431,0.030078
5,1010.872500,11.055098,0.000019,1.600657e-05,9.370015e-06,-4.528292e-07,2.002809e-06,1.070390e-06,334.466675,350.560791,...,-6.608168e-03,-0.017714,-0.021627,-0.035596,-0.043222,-0.056014,-0.044874,-0.014129,0.008889,0.016563
6,1009.393125,9.698686,0.000015,1.280468e-05,4.133205e-06,-2.130542e-06,-3.962862e-06,1.949821e-06,334.897827,350.389313,...,-3.783900e-02,-0.048879,-0.046826,-0.050394,-0.056823,-0.072708,-0.067299,-0.036301,-0.013679,-0.005593
7,1008.601875,10.552703,0.000025,1.823928e-05,3.255444e-06,-2.455291e-06,-3.976239e-06,3.579342e-06,335.233459,350.222290,...,-3.448741e-02,-0.044613,-0.044417,-0.045146,-0.054961,-0.068550,-0.055472,-0.021355,0.001744,0.009376
8,1009.268125,9.885348,0.000017,2.715001e-05,2.165952e-05,8.255443e-06,-8.395885e-06,-4.592875e-06,334.793732,350.052917,...,-2.344829e-02,-0.033209,-0.035389,-0.038074,-0.044089,-0.050432,-0.039264,-0.007122,0.015637,0.025487
9,1011.303750,10.535303,0.000023,3.146631e-05,2.751788e-05,1.132371e-05,-7.554091e-06,-4.758706e-06,334.121826,350.374084,...,-3.446163e-02,-0.047999,-0.055526,-0.056832,-0.052626,-0.046320,-0.036318,-0.017302,-0.003794,0.002556


In [49]:
df1=ts_bulbul
df1=df1.drop('ws10', axis=1)
df1=df1.drop('u10', axis=1)
df1=df1.drop('v10', axis=1)
df1=df1.drop('out_ws10', axis=1)
df1=df1.drop('outu10', axis=1)
df1=df1.drop('outv10', axis=1)
df1=df1.drop('vi_div_ke_flux', axis=1)
df1=df1.drop('vi_div_gpot_flux', axis=1)
df1=df1.drop('vi_ke', axis=1)
df1=df1.drop('vi_div_olr_flux', axis=1)
df1=df1.drop('vi_temp', axis=1)
df1=df1.drop('inst_10m_wnd_gst', axis=1)
df1=df1.drop('surfmean_swr_flux', axis=1)
df1=df1.drop('surfmean_lwr_flux', axis=1)
df1=df1.drop('dwnwrdmean_swr_flux', axis=1)
df1=df1.drop('topmean_lwr_flux', axis=1)
df1=df1.drop('topmean_swr_flux', axis=1)
df1=df1.drop('outvi_div_ke_flux', axis=1)
df1=df1.drop('outvi_div_gpot_flux', axis=1)
df1=df1.drop('outvi_ke', axis=1)
df1=df1.drop('outvi_div_olr_flux', axis=1)
df1=df1.drop('outvi_temp', axis=1)
df1=df1.drop('outinst_10m_wnd_gst', axis=1)
df1=df1.drop('outsurfmean_swr_flux', axis=1)
df1=df1.drop('outsurfmean_lwr_flux', axis=1)
df1=df1.drop('outdwnwrdmean_swr_flux', axis=1)
df1=df1.drop('outtopmean_lwr_flux', axis=1)
df1=df1.drop('outtopmean_swr_flux', axis=1)
df1=df1.drop('outmslp', axis=1)
df1=df1.drop('mslp', axis=1)
df1

,Pmin,w10max,div100,div200,div250,div300,div400,div500,eqt1000,eqt200,...,outvvel_250,outvvel_300,outvvel_400,outvvel_500,outvvel_600,outvvel_700,outvvel_850,outvvel_925,outvvel_975,outvvel_1000
0,1009.906250,10.584517,0.000004,1.360982e-05,4.113630e-06,-3.951576e-06,-1.947213e-06,-5.324125e-06,332.753601,349.601562,...,-6.098684e-03,-0.014341,-0.008909,-0.010385,-0.015297,-0.024262,-0.022208,0.000484,0.022405,0.031502
1,1011.211250,10.558450,-0.000007,1.180217e-05,5.048701e-06,7.306909e-07,2.372099e-07,7.937452e-07,328.466949,349.497437,...,-1.208216e-02,-0.019809,-0.015735,-0.027773,-0.030505,-0.037675,-0.034355,-0.017912,-0.008622,-0.005007
2,1009.446875,11.040154,-0.000008,1.765924e-05,1.021891e-05,4.357842e-06,1.986422e-06,1.384878e-06,326.333008,349.679840,...,-1.101356e-02,-0.023957,-0.037064,-0.062312,-0.065495,-0.066358,-0.055082,-0.043915,-0.040607,-0.041222
3,1007.248750,10.997210,0.000002,1.242364e-05,-3.431449e-06,-7.650060e-06,9.642074e-07,4.334469e-06,327.476135,350.137573,...,-4.615784e-03,-0.020518,-0.038315,-0.062682,-0.062309,-0.057834,-0.035955,-0.014446,-0.005745,-0.003362
4,1008.545000,10.445864,0.000016,1.031823e-05,3.785833e-06,-3.954945e-06,2.207007e-06,6.988461e-06,330.158203,350.548615,...,-1.248678e-03,-0.013755,-0.025165,-0.045954,-0.049125,-0.046761,-0.021107,0.007155,0.024431,0.030078
5,1010.872500,11.055098,0.000019,1.600657e-05,9.370015e-06,-4.528292e-07,2.002809e-06,1.070390e-06,334.466675,350.560791,...,-6.608168e-03,-0.017714,-0.021627,-0.035596,-0.043222,-0.056014,-0.044874,-0.014129,0.008889,0.016563
6,1009.393125,9.698686,0.000015,1.280468e-05,4.133205e-06,-2.130542e-06,-3.962862e-06,1.949821e-06,334.897827,350.389313,...,-3.783900e-02,-0.048879,-0.046826,-0.050394,-0.056823,-0.072708,-0.067299,-0.036301,-0.013679,-0.005593
7,1008.601875,10.552703,0.000025,1.823928e-05,3.255444e-06,-2.455291e-06,-3.976239e-06,3.579342e-06,335.233459,350.222290,...,-3.448741e-02,-0.044613,-0.044417,-0.045146,-0.054961,-0.068550,-0.055472,-0.021355,0.001744,0.009376
8,1009.268125,9.885348,0.000017,2.715001e-05,2.165952e-05,8.255443e-06,-8.395885e-06,-4.592875e-06,334.793732,350.052917,...,-2.344829e-02,-0.033209,-0.035389,-0.038074,-0.044089,-0.050432,-0.039264,-0.007122,0.015637,0.025487
9,1011.303750,10.535303,0.000023,3.146631e-05,2.751788e-05,1.132371e-05,-7.554091e-06,-4.758706e-06,334.121826,350.374084,...,-3.446163e-02,-0.047999,-0.055526,-0.056832,-0.052626,-0.046320,-0.036318,-0.017302,-0.003794,0.002556


In [50]:
df1.to_csv(output+'timeseries_all_nio_bulbul.csv')

In [51]:
#storms 7

mainpawan=pd.read_csv(output+'inner/inner_nio_all_pawan.csv',delimiter=r",")
mainpawan.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
mainpawan=mainpawan.drop('a', axis=1)

pdf_pawan=pd.read_csv(target+'2019_pmin_nio_pawan.csv',delimiter=r",")
pdf_pawan.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
pdf_pawan=pdf_pawan.drop('a', axis=1)

extra=pd.read_csv(output+'nio/outder_2019_nio_pawan.csv',delimiter=r",")
extra.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
extra=extra.drop('a', axis=1)


slev_pawan=pd.DataFrame.from_dict(TCs_ts[6])
rlev_pawan=pd.DataFrame.from_dict(TCr_ts[6])
prlev_pawan=pd.DataFrame.from_dict(TCpr_ts[6])
prlev_pawan['outmslp']=prlev_pawan['outmslp']/100

ts1_pawan=pd.concat([mainpawan,extra,slev_pawan,rlev_pawan,prlev_pawan], axis=1, join='inner')

tempvortdict = {'outvort_'+plev[i]:TCp_ts[6]['outvort'][:,i] for i in range(21)}
temprhumdict = {'outrhum_'+plev[i]:TCp_ts[6]['outrhum'][:,i] for i in range(21)}
tempgpotdict = {'outgpot_'+plev[i]:TCgp_ts[6]['outgpot'][:,i] for i in range(21)}
tempvveldict = {'outvvel_'+preslv[i]:TCvv_ts[6]['outvvel'][:,i] for i in range(18)}
dict1 = {**tempvortdict,**temprhumdict}
dict3 = {**tempgpotdict,**tempvveldict}

alldict1 = {**dict1,**dict3}

plev_pawan=pd.DataFrame.from_dict(alldict1)

ts_pawan=pd.concat([pdf_pawan,ts1_pawan,plev_pawan], axis=1, join='inner')
ts_pawan


,Pmin,w10max,div100,div200,div250,div300,div400,div500,eqt1000,eqt200,...,outvvel_250,outvvel_300,outvvel_400,outvvel_500,outvvel_600,outvvel_700,outvvel_850,outvvel_925,outvvel_975,outvvel_1000
0,1008.056875,14.006958,0.000011,0.000013,-1.946460e-07,-9.440578e-06,-2.539376e-06,8.448318e-07,334.220917,350.382416,...,-0.083361,-0.075170,-0.049873,-0.058831,-0.043738,-0.030314,-0.029546,-0.024287,-0.014811,-0.008148
1,1006.026875,13.472538,0.000017,0.000015,-6.533957e-06,-1.114228e-05,-3.305323e-06,2.794746e-06,332.563965,350.115356,...,-0.055358,-0.049359,-0.038071,-0.057226,-0.051911,-0.045619,-0.042629,-0.036334,-0.023865,-0.016097
2,1005.286875,12.542070,0.000018,0.000008,-2.292947e-06,-6.008996e-06,-8.418807e-06,9.971647e-06,335.469788,349.781097,...,-0.055858,-0.052979,-0.027940,-0.049288,-0.038015,-0.031531,-0.028355,-0.021021,-0.003791,0.006673
3,1005.915000,12.942327,0.000012,0.000010,3.175745e-07,1.459453e-07,-2.392968e-07,1.176803e-05,334.893250,349.381409,...,-0.041471,-0.039766,-0.021685,-0.039721,-0.037240,-0.040220,-0.033076,-0.020660,0.002790,0.016491
4,1006.827500,13.066185,0.000010,0.000023,4.677587e-06,-5.248843e-06,-1.415992e-06,9.797772e-06,328.935272,349.696594,...,-0.031308,-0.036601,-0.038123,-0.059305,-0.067613,-0.080540,-0.068032,-0.052831,-0.028164,-0.014427
5,1004.420000,13.048920,0.000011,0.000016,3.257270e-06,-7.133162e-06,2.574081e-06,5.239961e-06,330.978882,350.597961,...,-0.040995,-0.050183,-0.060816,-0.098912,-0.111841,-0.123788,-0.095861,-0.072266,-0.045796,-0.033064
6,1003.585625,12.942945,0.000009,0.000015,-1.096919e-06,-6.769892e-06,3.612737e-06,-2.491233e-06,334.835541,351.017822,...,-0.068403,-0.073319,-0.050682,-0.076896,-0.075414,-0.077770,-0.044097,-0.025138,-0.005522,0.003629
7,1004.741875,12.806230,0.000022,0.000010,2.474635e-06,-4.936801e-06,1.200215e-05,4.143376e-06,336.082214,351.615509,...,-0.078346,-0.080871,-0.058412,-0.073941,-0.058545,-0.055120,-0.030610,-0.014089,0.003075,0.011342
8,1005.268125,12.494928,0.000036,0.000023,5.647308e-06,-1.861424e-06,4.411397e-06,2.373149e-06,336.463989,351.797394,...,-0.087173,-0.094144,-0.078058,-0.087897,-0.067751,-0.064505,-0.049521,-0.035404,-0.020628,-0.012768
9,1003.003125,13.289216,0.000052,0.000028,-4.657593e-06,-2.216604e-06,2.622066e-06,-5.397631e-06,336.556824,352.464539,...,-0.090008,-0.103746,-0.095815,-0.101359,-0.088041,-0.087734,-0.066547,-0.044411,-0.023959,-0.012992


In [52]:
df1=ts_pawan
df1=df1.drop('ws10', axis=1)
df1=df1.drop('u10', axis=1)
df1=df1.drop('v10', axis=1)
df1=df1.drop('out_ws10', axis=1)
df1=df1.drop('outu10', axis=1)
df1=df1.drop('outv10', axis=1)
df1=df1.drop('vi_div_ke_flux', axis=1)
df1=df1.drop('vi_div_gpot_flux', axis=1)
df1=df1.drop('vi_ke', axis=1)
df1=df1.drop('vi_div_olr_flux', axis=1)
df1=df1.drop('vi_temp', axis=1)
df1=df1.drop('inst_10m_wnd_gst', axis=1)
df1=df1.drop('surfmean_swr_flux', axis=1)
df1=df1.drop('surfmean_lwr_flux', axis=1)
df1=df1.drop('dwnwrdmean_swr_flux', axis=1)
df1=df1.drop('topmean_lwr_flux', axis=1)
df1=df1.drop('topmean_swr_flux', axis=1)
df1=df1.drop('outvi_div_ke_flux', axis=1)
df1=df1.drop('outvi_div_gpot_flux', axis=1)
df1=df1.drop('outvi_ke', axis=1)
df1=df1.drop('outvi_div_olr_flux', axis=1)
df1=df1.drop('outvi_temp', axis=1)
df1=df1.drop('outinst_10m_wnd_gst', axis=1)
df1=df1.drop('outsurfmean_swr_flux', axis=1)
df1=df1.drop('outsurfmean_lwr_flux', axis=1)
df1=df1.drop('outdwnwrdmean_swr_flux', axis=1)
df1=df1.drop('outtopmean_lwr_flux', axis=1)
df1=df1.drop('outtopmean_swr_flux', axis=1)
df1=df1.drop('outmslp', axis=1)
df1=df1.drop('mslp', axis=1)
df1

,Pmin,w10max,div100,div200,div250,div300,div400,div500,eqt1000,eqt200,...,outvvel_250,outvvel_300,outvvel_400,outvvel_500,outvvel_600,outvvel_700,outvvel_850,outvvel_925,outvvel_975,outvvel_1000
0,1008.056875,14.006958,0.000011,0.000013,-1.946460e-07,-9.440578e-06,-2.539376e-06,8.448318e-07,334.220917,350.382416,...,-0.083361,-0.075170,-0.049873,-0.058831,-0.043738,-0.030314,-0.029546,-0.024287,-0.014811,-0.008148
1,1006.026875,13.472538,0.000017,0.000015,-6.533957e-06,-1.114228e-05,-3.305323e-06,2.794746e-06,332.563965,350.115356,...,-0.055358,-0.049359,-0.038071,-0.057226,-0.051911,-0.045619,-0.042629,-0.036334,-0.023865,-0.016097
2,1005.286875,12.542070,0.000018,0.000008,-2.292947e-06,-6.008996e-06,-8.418807e-06,9.971647e-06,335.469788,349.781097,...,-0.055858,-0.052979,-0.027940,-0.049288,-0.038015,-0.031531,-0.028355,-0.021021,-0.003791,0.006673
3,1005.915000,12.942327,0.000012,0.000010,3.175745e-07,1.459453e-07,-2.392968e-07,1.176803e-05,334.893250,349.381409,...,-0.041471,-0.039766,-0.021685,-0.039721,-0.037240,-0.040220,-0.033076,-0.020660,0.002790,0.016491
4,1006.827500,13.066185,0.000010,0.000023,4.677587e-06,-5.248843e-06,-1.415992e-06,9.797772e-06,328.935272,349.696594,...,-0.031308,-0.036601,-0.038123,-0.059305,-0.067613,-0.080540,-0.068032,-0.052831,-0.028164,-0.014427
5,1004.420000,13.048920,0.000011,0.000016,3.257270e-06,-7.133162e-06,2.574081e-06,5.239961e-06,330.978882,350.597961,...,-0.040995,-0.050183,-0.060816,-0.098912,-0.111841,-0.123788,-0.095861,-0.072266,-0.045796,-0.033064
6,1003.585625,12.942945,0.000009,0.000015,-1.096919e-06,-6.769892e-06,3.612737e-06,-2.491233e-06,334.835541,351.017822,...,-0.068403,-0.073319,-0.050682,-0.076896,-0.075414,-0.077770,-0.044097,-0.025138,-0.005522,0.003629
7,1004.741875,12.806230,0.000022,0.000010,2.474635e-06,-4.936801e-06,1.200215e-05,4.143376e-06,336.082214,351.615509,...,-0.078346,-0.080871,-0.058412,-0.073941,-0.058545,-0.055120,-0.030610,-0.014089,0.003075,0.011342
8,1005.268125,12.494928,0.000036,0.000023,5.647308e-06,-1.861424e-06,4.411397e-06,2.373149e-06,336.463989,351.797394,...,-0.087173,-0.094144,-0.078058,-0.087897,-0.067751,-0.064505,-0.049521,-0.035404,-0.020628,-0.012768
9,1003.003125,13.289216,0.000052,0.000028,-4.657593e-06,-2.216604e-06,2.622066e-06,-5.397631e-06,336.556824,352.464539,...,-0.090008,-0.103746,-0.095815,-0.101359,-0.088041,-0.087734,-0.066547,-0.044411,-0.023959,-0.012992


In [53]:
df1.to_csv(output+'timeseries_all_nio_pawan.csv')